In [ ]:
import os 
import urllib 
import shutil
import azureml

from azureml.core import Experiment
from azureml.core import Workspace, Run
from azureml.core import Environment, ScriptRunConfig

from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

from azureml.core.dataset import Dataset

from azureml.core import Model
from azureml.core.resource_configuration import ResourceConfiguration
from azureml.tensorboard import Tensorboard

In [ ]:
ws = Workspace.from_config()
dataset = ws.datasets["Refactored Classification Dataset"]
mount_ctx = dataset.as_mount()  
# dataset = Dataset.get_by_name(ws, name='Refactored Classification Dataset')
# dataset.download(target_path='.', overwrite=False)


In [ ]:
cluster_name = "gpu-cluster"
compute_target = ComputeTarget(workspace=ws, name=cluster_name)
print('Found existing compute target')

In [ ]:
newenv = Environment.get(workspace=ws, name="Custom_tensorflow_2_4")

In [ ]:
args = ['--data-folder', mount_ctx,
        '--batch-size', 64,
        '--first-layer-neurons', 256,
        '--second-layer-neurons', 128,
        '--learning-rate', 0.01]

src = ScriptRunConfig(source_directory="./training_files",
                      script='pneumothorax_classification.py',
                      arguments=args,
                      compute_target=compute_target,
                      environment=newenv)

In [ ]:
run = Experiment(workspace=ws, name='vggbased_aug_fine_tuning').submit(src)
tb = Tensorboard([run])
tb.start()
run.wait_for_completion(show_output=True)
tb.stop()

In [ ]:
model = run.register_model(model_name='vggbased_aug_fine_tuned', 
                           model_path='outputs/',
                           model_framework=Model.Framework.TENSORFLOW,
                           model_framework_version='2.0',
                           resource_configuration=ResourceConfiguration(cpu=1, memory_in_gb=0.5))